In [12]:
########
##
## パラメータ設定セル
##
########
# hostlistから対象hostをfilterし、指定したpackageをupgradeするためのinventoryファイルとplaybookを出力する。
# ファイルの出力先は、package_man/playbook/upgrade_package_playbookディレクトリ。
# **kwargsで指定できるkey(列名)は、hostlistの"ssh_ip_address"と各列。
# self.df.columnsの各要素は、hostlist.pyでhostを登録する際にセットしたtag(key-value)により、変動する。
# 例
# instance.make_package_upgrade_playbook(ssh_ip_address="192.168.0.100", package_manager="apt")
# self.dfのindexであるssh_ip_addressでfilterすると、1台のホストのみにfilterされる。
# post_upgrade_taskは、upgrade後の対象ホストの処理について。指定なし|reloaded|restarted|rebootのいずれか
# service_nameは、対象パッケージアップグレード後にreloadまたはrestartするservice名を指定。パラメータはpost_upgrade_taskがreloaded|restartedの場合のみ有効。指定なし|<reload,またはrestartするservice名>。

package = "python3-venv"
# post_upgrade_task = "reloaded"|"restarted"
# service_name = "<service>" -> "sshd"とか、"httpd"とか。
# 以下は、作成したhostlistのタグ名でfilter(抽出)する
# env = "dev", package_manager="apt"など。hostlist.pyの任意タグとタグ値でfilter可能。

In [13]:
import os, sys
from pathlib import Path

CURRENT_DIR = Path(os.getcwd())
PJROOT_DIR = CURRENT_DIR.parent
LIB_PACKAGE_MAN_DIR = PJROOT_DIR / "airflow"

sys.path.append(str(LIB_PACKAGE_MAN_DIR))

In [14]:
from lib_package_man import MakePlaybook, PackageList

In [15]:
# upgrade対象ホストの確認1
# 指定したパッケージがインストールされたホスト一覧
ins_PL = PackageList()
ins_PL.get_unique_hostlist_df(name=package)

Index(['ssh_ip_address', 'ssh_auth_method', 'ssh_user', 'ssh_password',
       'ssh_secretkey_path', 'name', 'package_manager', 'env'],
      dtype='object')


,ssh_ip_address,ssh_auth_method,ssh_user,ssh_password,ssh_secretkey_path,name,package_manager,env
1,192.168.0.101,password,$ANSIBLE_USER,$ANSIBLE_SSH_PASS,NaN,stdb1,apt,dev


In [11]:
# hostlistから対象hostをfilterし、指定したpackageをupgradeするためのinventoryファイルとplaybookを出力する。
# ファイルの出力先は、package_man/playbook/upgrade_package_playbookディレクトリ。
# upgradeは、playbook出力先フォルダに移動し、ansible-playbookコマンドを実行することで行う。

# 実行コマンド
# STEP1: playbook出力先フォルダに移動
# $ cd package_man/playbook/upgrade_package_playbook
# STEP2: inventoryファイルでansibleが対象ホストに正しくSSHでログインできているか確認
# $ ansible all -i inventory.yml -m ping
# STEP3: packageのupgrade
# $ ansible-playbook -i inventory.yml upgrade-package.yml
ins_MP = MakePlaybook()
ins_MP.make_upgrade_package_inventory_and_playbook(package)
# ins_MP.make_upgrade_package_inventory_and_playbook(package, post_upgrade_task="", service_name="", **kwargs)

Call MakePlaybook.make_get_packages_playbook method
no filter data
Index(['ssh_ip_address', 'ssh_auth_method', 'ssh_user', 'ssh_password',
       'ssh_secretkey_path', 'name', 'package_manager', 'env'],
      dtype='object')
Playbook /home/ogs-digilife/package_man/playbook/upgrade_package_playbook/upgrade-package.yml was made
Call MakePlaybook.make_inventory method
Inventory file /home/ogs-digilife/package_man/playbook/upgrade_package_playbook/inventory.yml was made

